In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os
import joblib
import pickle
from mplsoccer.pitch import Pitch
from ipynb.fs.defs.ExpectedGoalsData import get_df_shots
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None

In [ ]:
plt.style.use('fivethirtyeight')
params = {'axes.titlesize': 'x-large',
          'axes.labelsize': 'large', 
          'xtick.labelsize':'large',
          'ytick.labelsize':'large'}
plt.rcParams.update(params)

In [2]:
shots = pd.read_excel('extracted_data/shots_pred.xlsx')
matches = pd.read_excel('extracted_data/matches.xlsx')
players =  pd.read_excel('extracted_data/players.xlsx')

In [3]:
def get_player_name(player_id):
    if pd.notna(players.loc[players['player_id'] == player_id, 'player_nickname'].iloc[0]):
        return players.loc[players['player_id'] == player_id, 'player_nickname'].iloc[0]
    else:
        return players.loc[players['player_id'] == player_id, 'player_name'].iloc[0]

In [4]:
def get_competition_shots(competition_id):
    comp_matches = matches[matches['competition_id'] == competition_id]
    events_folder = 'data/events'
    events = pd.concat([pd.DataFrame(json.load(open(os.path.join(events_folder, str(match_id) + '.json'), encoding='utf8'))).assign(match_id=match_id) for match_id in comp_matches['match_id'].unique()])
    events.reset_index(inplace=True)
    comp_shots = get_df_shots(events)
    xg_model = joblib.load('models/expected_goals_model.pkl')
    with open("models/xg_selected_features.txt", "rb") as fp:
         xg_selected_features = pickle.load(fp)
    comp_shots.loc[:, 'pred_xg'] = xg_model.predict_proba(comp_shots[xg_selected_features])[:, 1]
    return comp_shots

In [6]:
# performance evaluation of goalkeepers
def rate_goalkeepers(all_shots):
    df_goalkeepers = pd.DataFrame()
    goalkeeper_ids = all_shots['goalkeeper_id'].unique()
    goalkeeper_ids = goalkeeper_ids[goalkeeper_ids != -1]
    df_goalkeepers.loc[:, 'goalkeeper_id'] = goalkeeper_ids
    df_goalkeepers.loc[:, 'Torwart'] = [get_player_name(gk_id) for gk_id in goalkeeper_ids]
    df_goalkeepers.loc[:, 'Torschüsse'] = [len(all_shots[all_shots['goalkeeper_id'] == gk_id]) for gk_id in goalkeeper_ids]
    df_goalkeepers.loc[:, 'xG/Torschuss'] = [np.round(all_shots.loc[all_shots['goalkeeper_id'] == gk_id, 'pred_xg'].mean(), 2) for gk_id in goalkeeper_ids]
    df_goalkeepers.loc[:, 'Expected Goals'] = [np.round(all_shots.loc[all_shots['goalkeeper_id'] == gk_id, 'pred_xg'].sum(), 2) for gk_id in goalkeeper_ids]
    df_goalkeepers.loc[:, 'Gegentore'] = [len(all_shots[(all_shots['goalkeeper_id'] == gk_id) & (all_shots['goal'] == 1)]) for gk_id in goalkeeper_ids]
    df_goalkeepers.loc[:, 'Torverhinderung (%)'] = np.round(100 * (df_goalkeepers['Expected Goals'] - df_goalkeepers['Gegentore']) / df_goalkeepers['Expected Goals'], 2)
    return df_goalkeepers

In [7]:
# performance evaluation of player's shots
def rate_player_shots(all_shots):
    df_players = pd.DataFrame()
    player_ids = all_shots['player_id'].unique()
    df_players.loc[:, 'player_id'] = player_ids
    df_players.loc[:, 'Spieler'] = [get_player_name(player_id) for player_id in player_ids]
    df_players.loc[:, 'Torschüsse'] = [len(all_shots[all_shots['player_id'] == player_id]) for player_id in player_ids]
    df_players.loc[:, 'xG/Torschuss'] = [np.round(all_shots.loc[all_shots['player_id'] == player_id, 'pred_xg'].mean(), 2) for player_id in player_ids]
    df_players.loc[:, 'Expected Goals'] = [np.round(all_shots.loc[all_shots['player_id'] == player_id, 'pred_xg'].sum(), 2) for player_id in player_ids]
    df_players.loc[:, 'Tore'] = [len(all_shots[(all_shots['player_id'] == player_id) & (all_shots['goal'] == 1)]) for player_id in player_ids]
    df_players.loc[:, 'Trefferquote (%)'] = np.round(100 * df_players['Tore'] / df_players['Expected Goals'], 2)
    return df_players

In [8]:
# performance evaluation of player's assists
def rate_player_assists(all_shots):
    df_assists = pd.DataFrame()
    player_ids = all_shots['player_id'].unique()
    df_assists.loc[:, 'player_id'] = player_ids
    df_assists.loc[:, 'Spieler'] = [get_player_name(player_id) for player_id in player_ids]
    df_assists.loc[:, 'Torschussvorlagen'] = [len(all_shots[all_shots['assistant_id'] == player_id]) for player_id in player_ids]
    df_assists.loc[:, 'xG/Vorlage'] = [np.round(all_shots.loc[all_shots['assistant_id'] == player_id, 'pred_xg'].mean(), 2) for player_id in player_ids]
    df_assists.loc[:, 'Expected Assists'] = [np.round(all_shots.loc[all_shots['assistant_id'] == player_id, 'pred_xg'].sum(), 2) for player_id in player_ids]
    df_assists.loc[:, 'Assists'] = [len(all_shots[(all_shots['assistant_id'] == player_id) & (all_shots['goal'] == 1)]) for player_id in player_ids]
    df_assists.loc[:, 'Assistquote (%)'] = np.round(100 * df_assists['Assists'] / df_assists['Expected Assists'], 2)
    return df_assists

In [9]:
# performance evaluation of teams
def rate_team_shots(all_shots):
    df_teams = pd.DataFrame()
    team_ids = all_shots['team_id'].unique()
    df_teams.loc[:, 'team_id'] = team_ids
    df_teams.loc[:, 'Team'] = [matches.loc[matches['away_team_id'] == team_id, 'away_team_name'].iloc[0] if len(matches[matches['away_team_id'] == team_id]) > 0 else '' for team_id in team_ids]
    df_teams.loc[:, 'Torschüsse'] = [len(all_shots[all_shots['team_id'] == team_id]) for team_id in team_ids]
    df_teams.loc[:, 'xG/Torschuss'] = [np.round(all_shots.loc[all_shots['team_id'] == team_id, 'pred_xg'].mean(), 2) for team_id in team_ids]
    df_teams.loc[:, 'Expected Goals'] = [np.round(all_shots.loc[all_shots['team_id'] == team_id, 'pred_xg'].sum(), 2) for team_id in team_ids]
    df_teams.loc[:, 'Tore'] = [len(all_shots[(all_shots['team_id'] == team_id) & (all_shots['goal'] == 1)]) for team_id in team_ids]
    df_teams.loc[:, 'Trefferquote (%)'] = np.round(100 * df_teams['Tore'] / df_teams['Expected Goals'], 2)
    return df_teams

Performace Evaluation on the World Cup 2018

In [10]:
# get shot predictions for the world cup 2018
wc_shots = get_competition_shots(43)
# drop penalties from penalty shoutouts
wc_shots = wc_shots[(wc_shots['minute'] < 120) | (wc_shots['penalty'] == 0)]
# fill missing goalkeeper_ids
for idx, row in wc_shots[wc_shots['goalkeeper_id'] == -1].iterrows():
    wc_shots.loc[idx, 'goalkeeper_id'] = wc_shots.loc[(wc_shots['match_id'] == row['match_id']) & (wc_shots['team_id'] == row['team_id']), 'goalkeeper_id'].max()

ipynb.fs.defs.ExpectedGoalsData:117: RuntimeWarning: divide by zero encountered in double_scalars


In [11]:
df_goalkeepers = rate_goalkeepers(wc_shots)
df_goalkeepers[df_goalkeepers['Expected Goals'] >= 6].sort_values(by='Torverhinderung (%)', ascending=False)

,goalkeeper_id,Torwart,Torschüsse,xG/Torschuss,Expected Goals,Gegentore,Torverhinderung (%)
1,3815,Kasper Schmeichel,62,0.12,7.40,2,72.97
30,5619,Cho Hyun-Woo,59,0.14,8.07,3,62.83
4,4276,David Ospina,49,0.14,6.92,3,56.65
9,5577,Guillermo Ochoa,77,0.15,11.31,5,55.79
20,3509,Thibaut Courtois,107,0.11,11.39,6,47.32
16,5550,Yann Sommer,59,0.12,7.23,4,44.67
18,5172,Igor Akinfeev,79,0.11,8.66,5,42.26
6,5597,Keylor Navas,48,0.15,6.96,5,28.16
39,3468,Jordan Pickford,83,0.13,10.91,8,26.67
21,3175,Eiji Kawashima,64,0.14,9.00,7,22.22


In [12]:
df_players = rate_player_shots(wc_shots)
df_players[df_players['Tore'] >= 2].sort_values(by='Trefferquote (%)', ascending=False)

,player_id,Spieler,Torschüsse,xG/Torschuss,Expected Goals,Tore,Trefferquote (%)
36,6196,Yerry Mina,3,0.15,0.46,3,652.17
22,5473,Ahmed Musa,4,0.15,0.61,2,327.87
138,5186,Denis Cheryshev,13,0.11,1.42,4,281.69
183,5687,Takashi Inui,9,0.09,0.79,2,253.16
201,3237,Sergio Agüero,6,0.17,1.05,2,190.48
144,5195,Artem Dzyuba,8,0.21,1.65,3,181.82
128,3009,Kylian Mbappé,8,0.28,2.22,4,180.18
350,3244,John Stones,6,0.20,1.21,2,165.29
349,10955,Harry Kane,14,0.31,4.35,6,137.93
141,4319,Edinson Cavani,16,0.14,2.21,3,135.75


In [13]:
df_assists = rate_player_assists(wc_shots)
df_assists[(df_assists['Assists'] >= 1) & (df_assists['Expected Assists'] >= 1)].sort_values(by='Assistquote (%)')

,player_id,Spieler,Torschussvorlagen,xG/Vorlage,Expected Assists,Assists,Assistquote (%)
54,4320,Neymar,24,0.13,3.16,1,31.65
354,3308,Kieran Trippier,25,0.10,2.49,1,40.16
193,5630,Dries Mertens,9,0.20,1.84,1,54.35
202,5503,Lionel Messi,9,0.20,1.84,1,54.35
53,3472,Willian,5,0.29,1.45,1,68.97
181,3089,Kevin De Bruyne,21,0.14,2.88,2,69.44
252,5216,Andrés Iniesta,7,0.19,1.36,1,73.53
92,3831,Dušan Tadić,8,0.16,1.25,1,80.00
250,5203,Sergio Busquets,3,0.41,1.24,1,80.65
361,5613,Viktor Claesson,7,0.17,1.22,1,81.97


In [14]:
df_teams = rate_team_shots(wc_shots)
df_teams[df_teams['Expected Goals'] >= 5].sort_values(by='Trefferquote (%)', ascending=False)

,team_id,Team,Torschüsse,xG/Torschuss,Expected Goals,Tore,Trefferquote (%)
19,796,Russia,50,0.11,5.74,10,174.22
17,771,France,82,0.14,11.34,12,105.82
28,780,Portugal,58,0.10,5.75,6,104.35
20,778,Japan,44,0.13,5.86,6,102.39
1,785,Croatia,114,0.12,13.35,13,97.38
21,782,Belgium,108,0.14,15.62,15,96.03
30,768,England,96,0.14,13.17,12,91.12
14,777,Tunisia,38,0.15,5.87,5,85.18
18,783,Uruguay,63,0.12,7.62,6,78.74
22,779,Argentina,58,0.14,8.06,6,74.44


Simulation der Abschlusstabelle nach xG für die FA Women's Super League 2018/19

In [15]:
# get shot predictions for the league
league_matches = matches[(matches['competition_id'] == 37) & (matches['season'] == '2018/2019')]
events_folder = 'data/events'
events = pd.concat([pd.DataFrame(json.load(open(os.path.join(events_folder, str(match_id) + '.json'), encoding='utf8'))).assign(match_id=match_id) for match_id in league_matches['match_id'].unique()])
events.reset_index(inplace=True)
league_shots = get_df_shots(events)
xg_model = joblib.load('models/expected_goals_model.pkl')
with open("models/xg_selected_features.txt", "rb") as fp:
     xg_selected_features = pickle.load(fp)
league_shots.loc[:, 'pred_xg'] = xg_model.predict_proba(league_shots[xg_selected_features])[:, 1]

ipynb.fs.defs.ExpectedGoalsData:117: RuntimeWarning: divide by zero encountered in double_scalars


In [16]:
# create tables
df_table = pd.DataFrame(columns=['Pos', 'Verein', 'Sp', 'S', 'U', 'N', 'T', 'GT', 'TD', 'Pkte'])
df_table['Verein'] = league_matches['home_team_name'].unique()
df_table.fillna(0, inplace=True)
df_pred_table = pd.DataFrame(columns=['Pos', 'Verein', 'Sp', 'S', 'U', 'N', 'T', 'GT', 'TD', 'Pkte'])
df_pred_table['Verein'] = league_matches['home_team_name'].unique()
df_pred_table.fillna(0, inplace=True)

In [17]:
def add_missing_matches(df):
    # Chelsea FCW 8:0 Yeovil Town LFC
    # Birmingham City WFC 1:0 Brighton & Hove Albion WFC
    # Liverpool WFC 3:1 Everton
    df.loc[df['Verein'] == 'Chelsea FCW', 'Sp'] += 1
    df.loc[df['Verein'] == 'Chelsea FCW', 'T'] += 8
    df.loc[df['Verein'] == 'Chelsea FCW', 'S'] += 1
    df.loc[df['Verein'] == 'Chelsea FCW', 'Pkte'] += 3
    df.loc[df['Verein'] == 'Yeovil Town LFC', 'Sp'] += 1
    df.loc[df['Verein'] == 'Yeovil Town LFC', 'GT'] += 8
    df.loc[df['Verein'] == 'Yeovil Town LFC', 'N'] += 1
    df.loc[df['Verein'] == 'Birmingham City WFC', 'Sp'] += 1
    df.loc[df['Verein'] == 'Birmingham City WFC', 'T'] += 1
    df.loc[df['Verein'] == 'Birmingham City WFC', 'S'] += 1
    df.loc[df['Verein'] == 'Birmingham City WFC', 'Pkte'] += 3
    df.loc[df['Verein'] == 'Brighton & Hove Albion WFC', 'Sp'] += 1
    df.loc[df['Verein'] == 'Brighton & Hove Albion WFC', 'GT'] += 1
    df.loc[df['Verein'] == 'Brighton & Hove Albion WFC', 'N'] += 1
    df.loc[df['Verein'] == 'Liverpool WFC', 'Sp'] += 1
    df.loc[df['Verein'] == 'Liverpool WFC', 'T'] += 3
    df.loc[df['Verein'] == 'Liverpool WFC', 'GT'] += 1
    df.loc[df['Verein'] == 'Liverpool WFC', 'S'] += 1
    df.loc[df['Verein'] == 'Liverpool WFC', 'Pkte'] += 3
    df.loc[df['Verein'] == 'Everton LFC', 'Sp'] += 1
    df.loc[df['Verein'] == 'Everton LFC', 'T'] += 1
    df.loc[df['Verein'] == 'Everton LFC', 'GT'] += 3
    df.loc[df['Verein'] == 'Everton LFC', 'N'] += 1
    return df

In [18]:
# predict results for all matches
for match_id in league_matches['match_id'].unique():
    match = league_matches[league_matches['match_id'] == match_id]
    match_id = match['match_id'].iloc[0]
    team1_id = match['home_team_id'].iloc[0]
    team2_id = match['away_team_id'].iloc[0]
    team1_name = match['home_team_name'].iloc[0]
    team2_name = match['away_team_name'].iloc[0]
    home_score = match['home_score'].iloc[0]
    away_score = match['away_score'].iloc[0]
    # add the real result of the match
    df_table.loc[df_table['Verein'] == team1_name, 'Sp'] += 1
    df_table.loc[df_table['Verein'] == team2_name, 'Sp'] += 1
    df_table.loc[df_table['Verein'] == team1_name, 'T'] += home_score
    df_table.loc[df_table['Verein'] == team2_name, 'T'] += away_score
    df_table.loc[df_table['Verein'] == team1_name, 'GT'] += away_score
    df_table.loc[df_table['Verein'] == team2_name, 'GT'] += home_score
    if home_score == away_score:
        df_table.loc[df_table['Verein'] == team1_name, 'U'] += 1
        df_table.loc[df_table['Verein'] == team2_name, 'U'] += 1
        df_table.loc[df_table['Verein'] == team1_name, 'Pkte'] += 1
        df_table.loc[df_table['Verein'] == team2_name, 'Pkte'] += 1
    elif home_score > away_score:
        df_table.loc[df_table['Verein'] == team1_name, 'S'] += 1
        df_table.loc[df_table['Verein'] == team2_name, 'N'] += 1
        df_table.loc[df_table['Verein'] == team1_name, 'Pkte'] += 3
        df_table.loc[df_table['Verein'] == team2_name, 'Pkte'] += 0
    else:
        df_table.loc[df_table['Verein'] == team1_name, 'N'] += 1
        df_table.loc[df_table['Verein'] == team2_name, 'S'] += 1
        df_table.loc[df_table['Verein'] == team1_name, 'Pkte'] += 0
        df_table.loc[df_table['Verein'] == team2_name, 'Pkte'] += 3
    # predict the result of the match
    team1_xg = league_shots.loc[(league_shots['match_id'] == match_id) & (league_shots['team_id'] == team1_id), 'pred_xg'].sum()
    team2_xg = league_shots.loc[(league_shots['match_id'] == match_id) & (league_shots['team_id'] == team2_id), 'pred_xg'].sum()
    df_pred_table.loc[df_pred_table['Verein'] == team1_name, 'Sp'] += 1
    df_pred_table.loc[df_pred_table['Verein'] == team2_name, 'Sp'] += 1
    if np.abs(team1_xg - team2_xg) < 0.5:
        df_pred_table.loc[df_pred_table['Verein'] == team1_name, 'U'] += 1
        df_pred_table.loc[df_pred_table['Verein'] == team2_name, 'U'] += 1
        df_pred_table.loc[df_pred_table['Verein'] == team1_name, 'Pkte'] += 1
        df_pred_table.loc[df_pred_table['Verein'] == team2_name, 'Pkte'] += 1
    elif team1_xg > team2_xg:
        df_pred_table.loc[df_pred_table['Verein'] == team1_name, 'S'] += 1
        df_pred_table.loc[df_pred_table['Verein'] == team2_name, 'N'] += 1
        df_pred_table.loc[df_pred_table['Verein'] == team1_name, 'Pkte'] += 3
        df_pred_table.loc[df_pred_table['Verein'] == team2_name, 'Pkte'] += 0
    else:
        df_pred_table.loc[df_pred_table['Verein'] == team1_name, 'N'] += 1
        df_pred_table.loc[df_pred_table['Verein'] == team2_name, 'S'] += 1
        df_pred_table.loc[df_pred_table['Verein'] == team1_name, 'Pkte'] += 0
        df_pred_table.loc[df_pred_table['Verein'] == team2_name, 'Pkte'] += 3

           
# add the 3 missing matches
df_table = add_missing_matches(df_table)
df_pred_table = add_missing_matches(df_pred_table)

# add real number of goals
df_table['TD'] = df_table['T'] - df_table['GT']

# predict number of goals
for team_id in league_matches['home_team_id'].unique():
    team_name = league_matches.loc[league_matches['home_team_id'] == team_id, 'home_team_name'].iloc[0]
    team_matches = league_matches.loc[(league_matches['home_team_id'] == team_id) | (league_matches['away_team_id'] == team_id), 'match_id'].unique()
    df_pred_table.loc[df_pred_table['Verein'] == team_name, 'T'] = np.round(league_shots.loc[league_shots['team_id'] == team_id, 'pred_xg'].sum())
    df_pred_table.loc[df_pred_table['Verein'] == team_name, 'GT'] = np.round(league_shots.loc[(league_shots['match_id'].isin(team_matches)) & (league_shots['team_id'] != team_id), 'pred_xg'].sum())
    df_pred_table.loc[df_pred_table['Verein'] == team_name, 'TD'] = df_pred_table.loc[df_pred_table['Verein'] == team_name, 'T'].iloc[0] - df_pred_table.loc[df_pred_table['Verein'] == team_name, 'GT'].iloc[0]
df_pred_table['T'] = df_pred_table['T'].astype(int)
df_pred_table['GT'] = df_pred_table['GT'].astype(int)
df_pred_table['TD'] = df_pred_table['TD'].astype(int)
    
# sort real table
df_table.sort_values(by='Pkte', inplace=True, ascending=False)
df_table['Pos'] = np.arange(1, len(df_table)+1)
df_table.reset_index(inplace=True, drop=True)

# sort predicted table
df_pred_table.sort_values(by='Pkte', inplace=True, ascending=False)
df_pred_table['Pos'] = np.arange(1, len(df_pred_table)+1)
df_pred_table.reset_index(inplace=True, drop=True)

# add differences to real table
for team_name in league_matches['home_team_name'].unique():
    df_pred_table.loc[df_pred_table['Verein'] == team_name, 'P'] = df_table.loc[df_table['Verein'] == team_name, 'Pos'].iloc[0] - df_pred_table.loc[df_pred_table['Verein'] == team_name, 'Pos'].iloc[0]
    df_pred_table.loc[df_pred_table['Verein'] == team_name, 'Pk'] = df_pred_table.loc[df_pred_table['Verein'] == team_name, 'Pkte'].iloc[0] - df_table.loc[df_table['Verein'] == team_name, 'Pkte'].iloc[0]
df_pred_table = df_pred_table[['P', 'Pos', 'Verein', 'Sp', 'S', 'U', 'N', 'T', 'GT', 'TD', 'Pkte', 'Pk']]
df_pred_table['P'] = [str(int(df_pred_table['P'].iloc[i])) if df_pred_table['P'].iloc[i] <= 0 else '+' + str(int(df_pred_table['P'].iloc[i])) for i in range(len(df_pred_table))]
df_pred_table['Pk'] = [str(int(df_pred_table['Pk'].iloc[i])) if df_pred_table['Pk'].iloc[i] <= 0 else '+' + str(int(df_pred_table['Pk'].iloc[i])) for i in range(len(df_pred_table))]

In [19]:
df_table

,Pos,Verein,Sp,S,U,N,T,GT,TD,Pkte
0,1,Arsenal WFC,20,18,0,2,70,13,57,54
1,2,Manchester City WFC,20,14,5,1,53,17,36,47
2,3,Chelsea FCW,20,12,6,2,46,14,32,42
3,4,Birmingham City WFC,20,13,1,6,29,17,12,40
4,5,Reading WFC,20,8,3,9,33,30,3,27
5,6,Bristol City WFC,20,7,4,9,17,34,-17,25
6,7,West Ham United LFC,20,7,2,11,25,37,-12,23
7,8,Liverpool WFC,20,7,1,12,21,38,-17,22
8,9,Brighton & Hove Albion WFC,20,4,4,12,16,38,-22,16
9,10,Everton LFC,20,3,3,14,15,38,-23,12


In [20]:
df_pred_table

,P,Pos,Verein,Sp,S,U,N,T,GT,TD,Pkte,Pk
0,0,1,Arsenal WFC,20,18,2,0,60,17,43,56,+2
1,+1,2,Chelsea FCW,20,16,3,1,54,17,37,51,+9
2,-1,3,Manchester City WFC,20,14,4,2,55,28,27,46,-1
3,+1,4,Reading WFC,20,9,6,5,36,34,2,33,+6
4,-1,5,Birmingham City WFC,20,8,7,5,30,26,4,31,-9
5,+3,6,Brighton & Hove Albion WFC,20,6,5,9,26,32,-6,23,+7
6,+1,7,Liverpool WFC,20,6,4,10,24,34,-10,22,0
7,-1,8,West Ham United LFC,20,5,5,10,30,38,-8,20,-3
8,+1,9,Everton LFC,20,4,4,12,25,38,-13,16,+4
9,-4,10,Bristol City WFC,20,2,4,14,16,47,-31,10,-15
